In [1]:
import pathlib
import yaml
ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.safe_load(f)

In [25]:
# 3
DATA_PATH = ROOT / config['api']['DATA_PATH'] / 'seminar4'

query = "k_beauty | kbeauty"
type_query = "video"

# Для поиска видео с учетом даты:
publishedAfter = '2020-01-01T00:00:00Z'
publishedBefore = '2022-01-01T00:00:00Z'

In [26]:
# 3.1
class KeyStorage:
    """
    Class to use in iteration
    """

    def __init__(self, key_arr: list[str]):
        self.key_gen = iter(key_arr)

    def get_next_key(self):
        """
        Returns API key to use
        :return:
        """
        return next(self.key_gen)

key_storage = KeyStorage(config['api']['keys'])

In [29]:
# 3.2 Просмотр API ключа
API_KEY = key_storage.get_next_key()

In [30]:
#4
import googleapiclient.discovery as api

In [31]:
#5
api_service_name = "youtube"
api_version = "v3"
    
youtube = api.build(api_service_name,
                        api_version, 
                       developerKey = API_KEY)

In [8]:
#8
import pandas as pd

In [10]:
# 18.1
# Первый заход БЕЗ аргумента order для видео
request = youtube.search().list(
        part ="snippet",
        maxResults=50,
        q = query,
        regionCode ="RU",
        type = type_query
)
response = request.execute()  
   
df_supplemented = pd.json_normalize(response['items'])
   
    # Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            type = type_query,
            pageToken = response['nextPageToken']
        )
    response = request.execute()  

    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1
        
    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])

    

print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates(f'id.{type_query}Id'))}")

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Итерация БЕЗ включения аргумента order №6
Итерация БЕЗ включения аргумента order №7
Итерация БЕЗ включения аргумента order №8
Итерация БЕЗ включения аргумента order №9
Искомых объектов 1000000 а найденных БЕЗ включения каких-либо значений аргумента order 498


In [11]:
len(df_supplemented)

523

In [12]:
# Значения аргумента order:
order_list = ['date', 'rating', 'title', 'videoCount', 'viewCount']
order_list

['date', 'rating', 'title', 'videoCount', 'viewCount']

In [14]:
# 18.2
# Цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи:
i = 0
for order in order_list:
    # Для остановки алгоритма, если все искомые объекты найдены
    # БЕЗ включения каких-либо значений аргумента order (в т.ч. вообще БЕЗ них):
    if len(df_supplemented.drop_duplicates(f'id.{type_query}Id')) < response['pageInfo']['totalResults']:
        
        # Первый заход с каждым значение аргумента order:
        request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            type = type_query,
            order = order
        )
        response = request.execute()
    
        # Визуализация процесса:
        print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
        i += 1
    
        # Занесение собранных данных в итоговую таблицу df_supplemented:
        df = pd.json_normalize(response["items"])
        df_supplemented = pd.concat([df, df_supplemented])
        
        # Заходы с тем же значением аргумента order на следующие страницы:
        while ('nextPageToken' in response.keys())\
        & (len(df_supplemented.drop_duplicates(f'id.{type_query}Id')) < response['pageInfo']['totalResults'])\
        & (len(response["items"]) > 0):
        # второе условие -- для остановки алгоритма, если все искомые объекты найдены
        # БЕЗ какой-то из следующих страниц ( в т.ч. вообще БЕЗ них)
        # третье условие -- для остановки алгоритма, если предыдущая страница выдачи содержит 0 объектов    
        
            request = youtube.search().list(
                part ="snippet",
                maxResults=50,
                q = query,
                type = type_query,
                pageToken = response['nextPageToken'],
                order = order
            )
            response = request.execute()  
    
            # Визуализация процесса:
            print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
            
            i += 1
        
            # Занесение собранных данных в итоговую таблицу df_supplemented:
            df_additional = pd.json_normalize(response["items"])
            df_supplemented = pd.concat([df_supplemented, df_additional])
    
        # Сохранение составленных для каждого значения аргумента order таблиц в Excel -- на всякий случай:
        df_supplemented.to_excel(DATA_PATH / f'{type_query.capitalize()}_Sorted_by_{order}.xlsx')
    else:
        print('Все искомые объекты найдены БЕЗ включения некоторых значений аргумента order (в т.ч. вообще БЕЗ них)')

Итерация №0, "order" date, "items" 50
Итерация №1, "order" date, "items" 50
Итерация №2, "order" date, "items" 50
Итерация №3, "order" date, "items" 50
Итерация №4, "order" date, "items" 50
Итерация №5, "order" date, "items" 50
Итерация №6, "order" date, "items" 50
Итерация №7, "order" date, "items" 50
Итерация №8, "order" date, "items" 50
Итерация №9, "order" date, "items" 50
Итерация №10, "order" date, "items" 50
Итерация №11, "order" date, "items" 15
Итерация №12, "order" rating, "items" 50
Итерация №13, "order" rating, "items" 50
Итерация №14, "order" rating, "items" 50
Итерация №15, "order" rating, "items" 50
Итерация №16, "order" rating, "items" 50
Итерация №17, "order" rating, "items" 50
Итерация №18, "order" rating, "items" 50
Итерация №19, "order" rating, "items" 50
Итерация №20, "order" rating, "items" 50
Итерация №21, "order" rating, "items" 50
Итерация №22, "order" rating, "items" 50
Итерация №23, "order" rating, "items" 16
Итерация №24, "order" title, "items" 50
Итерация №

In [15]:
# 19
# Удаление дубликатов каналов (по их ID) и запись в Excel:
df_supplemented.index = range(1,len(df_supplemented)+1) # сквозной индекс для итоговой таблицы
df_supplemented = df_supplemented.drop_duplicates(f'id.{type_query}Id')
display(df_supplemented)


,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
1,youtube#searchResult,xUPXCyswbDfQ0O_cEo1y9jBHY-I,youtube#video,AOVTUmDU7Bs,2018-07-12T06:00:04Z,UCgln4bTPN2cXnAjXVgM-Cfw,Annoying Moments While Getting Ready ENG SUB •...,ANNOYING MOMENTS WHILE GETTING READY!! UGH!! M...,https://i.ytimg.com/vi/AOVTUmDU7Bs/default.jpg,120,90,https://i.ytimg.com/vi/AOVTUmDU7Bs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AOVTUmDU7Bs/hqdefault.jpg,480,360,DGG,none,2018-07-12T06:00:04Z
2,youtube#searchResult,ry0zlUewmQ1_rgpvH5UeqQjwyOE,youtube#video,oNJWU8cm0eU,2020-08-25T06:38:17Z,UChhOtjq-3QyyLmP2jv9amrg,Rhythm of K-Beauty,Rhythm of K-Beauty Want to learn about Korean ...,https://i.ytimg.com/vi/oNJWU8cm0eU/default.jpg,120,90,https://i.ytimg.com/vi/oNJWU8cm0eU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/oNJWU8cm0eU/hqdefault.jpg,480,360,Imagine Your Korea,none,2020-08-25T06:38:17Z
3,youtube#searchResult,FAWm2FVeXAKFMi8_98auJXmRjZo,youtube#video,fgeqSoLgccU,2018-07-10T06:00:30Z,UCgln4bTPN2cXnAjXVgM-Cfw,Short Hair VS Long Hair ENG SUB • dingo kbeauty,WORLD'S FIRST PROBLEM This is #1 thing I ask m...,https://i.ytimg.com/vi/fgeqSoLgccU/default.jpg,120,90,https://i.ytimg.com/vi/fgeqSoLgccU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/fgeqSoLgccU/hqdefault.jpg,480,360,DGG,none,2018-07-10T06:00:30Z
4,youtube#searchResult,6Lv9z7Od-sjYY5pxt54NruY3weY,youtube#video,gvsV9ds9uFk,2022-04-15T04:52:25Z,UCJ9E_Ob8i8-2cyd67mbkRfw,Pretty duo 💓 #lipbalm #kbeauty,,https://i.ytimg.com/vi/gvsV9ds9uFk/default.jpg,120,90,https://i.ytimg.com/vi/gvsV9ds9uFk/mqdefault.jpg,320,180,https://i.ytimg.com/vi/gvsV9ds9uFk/hqdefault.jpg,480,360,Kerbom3,none,2022-04-15T04:52:25Z
5,youtube#searchResult,a5HhfVq4RM98pKJBdn9oqGBOICE,youtube#video,o5hc2PaPFZ8,2021-09-24T16:54:57Z,UCJ9E_Ob8i8-2cyd67mbkRfw,#kajabeauty #kbeauty #kbeautyreview #review #a...,,https://i.ytimg.com/vi/o5hc2PaPFZ8/default.jpg,120,90,https://i.ytimg.com/vi/o5hc2PaPFZ8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/o5hc2PaPFZ8/hqdefault.jpg,480,360,Kerbom3,none,2021-09-24T16:54:57Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,youtube#searchResult,yrLbtJF5skD__0R2g-TTUhPyM8c,youtube#video,DsOeHUYwjpw,2023-02-02T21:22:27Z,UCpjNgaKGfDCGPbP7JmgjZHw,Trying one of the TOP sunscreens in Korea 🫣 #k...,Follow for more glow ✨ Instagram: https://www....,https://i.ytimg.com/vi/DsOeHUYwjpw/default.jpg,120,90,https://i.ytimg.com/vi/DsOeHUYwjpw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/DsOeHUYwjpw/hqdefault.jpg,480,360,vpofglow,none,2023-02-02T21:22:27Z
3216,youtube#searchResult,xW4tKawA4C8AaI2gRu4UMfrhIIc,youtube#video,YGPP1BjvgCU,2023-02-02T10:48:45Z,UC8_stYIOMyTQs_cEKkopaFQ,"Let&#39;s do an Eye Mask, and Sheet Mask #revi...",Stylevana Makeup and Skincare. I have a code y...,https://i.ytimg.com/vi/YGPP1BjvgCU/default.jpg,120,90,https://i.ytimg.com/vi/YGPP1BjvgCU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/YGPP1BjvgCU/hqdefault.jpg,480,360,Life with Christine,none,2023-02-02T10:48:45Z
3269,youtube#searchResult,r1fvc4bljLvRM3LA8vuAlRthlKM,youtube#video,7JlZiPmfJUM,2023-02-20T08:03:00Z,UCnaasPYchRGCIsh9sCoTEhA,come on rest~ 4F solaria hotel #myeongdongkyoj...,,https://i.ytimg.com/vi/7JlZiPmfJUM/default.jpg,120,90,https://i.ytimg.com/vi/7JlZiPmfJUM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/7JlZiPmfJUM/hqdefault.jpg,480,360,MyeongDongHairSalon,none,2023-02-20T08:03:00Z
3281,youtube#searchResult,XqD468Q_82WhDFB0_OkgigCleIU,youtube#video,5c1f97i8XZM,2023-02-13T03:40:15Z,UC8J8CPTDZsFRh9dRfm-xayg,Are you into Vegan Products? #skincareroutine ...,"[ Unboxing ] Melixir, is a Vegan Skincare Bra

In [17]:
# 20

df_supplemented.to_excel(DATA_PATH /f'{query}_{type_query.capitalize()}_Not_sorted+Sorted.xlsx', index=False)

In [19]:
df_supplemented = pd.read_excel(DATA_PATH / f'{type_query.capitalize()}_Not_sorted+Sorted.xlsx')
# возврат к выдаче без учета date; указание, что индекс в первом столбце
df_supplemented

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,xUPXCyswbDfQ0O_cEo1y9jBHY-I,youtube#video,AOVTUmDU7Bs,2018-07-12T06:00:04Z,UCgln4bTPN2cXnAjXVgM-Cfw,Annoying Moments While Getting Ready ENG SUB •...,ANNOYING MOMENTS WHILE GETTING READY!! UGH!! M...,https://i.ytimg.com/vi/AOVTUmDU7Bs/default.jpg,120,90,https://i.ytimg.com/vi/AOVTUmDU7Bs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AOVTUmDU7Bs/hqdefault.jpg,480,360,DGG,none,2018-07-12T06:00:04Z
1,youtube#searchResult,ry0zlUewmQ1_rgpvH5UeqQjwyOE,youtube#video,oNJWU8cm0eU,2020-08-25T06:38:17Z,UChhOtjq-3QyyLmP2jv9amrg,Rhythm of K-Beauty,Rhythm of K-Beauty Want to learn about Korean ...,https://i.ytimg.com/vi/oNJWU8cm0eU/default.jpg,120,90,https://i.ytimg.com/vi/oNJWU8cm0eU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/oNJWU8cm0eU/hqdefault.jpg,480,360,Imagine Your Korea,none,2020-08-25T06:38:17Z
2,youtube#searchResult,FAWm2FVeXAKFMi8_98auJXmRjZo,youtube#video,fgeqSoLgccU,2018-07-10T06:00:30Z,UCgln4bTPN2cXnAjXVgM-Cfw,Short Hair VS Long Hair ENG SUB • dingo kbeauty,WORLD'S FIRST PROBLEM This is #1 thing I ask m...,https://i.ytimg.com/vi/fgeqSoLgccU/default.jpg,120,90,https://i.ytimg.com/vi/fgeqSoLgccU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/fgeqSoLgccU/hqdefault.jpg,480,360,DGG,none,2018-07-10T06:00:30Z
3,youtube#searchResult,6Lv9z7Od-sjYY5pxt54NruY3weY,youtube#video,gvsV9ds9uFk,2022-04-15T04:52:25Z,UCJ9E_Ob8i8-2cyd67mbkRfw,Pretty duo 💓 #lipbalm #kbeauty,NaN,https://i.ytimg.com/vi/gvsV9ds9uFk/default.jpg,120,90,https://i.ytimg.com/vi/gvsV9ds9uFk/mqdefault.jpg,320,180,https://i.ytimg.com/vi/gvsV9ds9uFk/hqdefault.jpg,480,360,Kerbom3,none,2022-04-15T04:52:25Z
4,youtube#searchResult,a5HhfVq4RM98pKJBdn9oqGBOICE,youtube#video,o5hc2PaPFZ8,2021-09-24T16:54:57Z,UCJ9E_Ob8i8-2cyd67mbkRfw,#kajabeauty #kbeauty #kbeautyreview #review #a...,NaN,https://i.ytimg.com/vi/o5hc2PaPFZ8/default.jpg,120,90,https://i.ytimg.com/vi/o5hc2PaPFZ8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/o5hc2PaPFZ8/hqdefault.jpg,480,360,Kerbom3,none,2021-09-24T16:54:57Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,youtube#searchResult,yrLbtJF5skD__0R2g-TTUhPyM8c,youtube#video,DsOeHUYwjpw,2023-02-02T21:22:27Z,UCpjNgaKGfDCGPbP7JmgjZHw,Trying one of the TOP sunscreens in Korea 🫣 #k...,Follow for more glow ✨ Instagram: https://www....,https://i.ytimg.com/vi/DsOeHUYwjpw/default.jpg,120,90,https://i.ytimg.com/vi/DsOeHUYwjpw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/DsOeHUYwjpw/hqdefault.jpg,480,360,vpofglow,none,2023-02-02T21:22:27Z
943,youtube#searchResult,xW4tKawA4C8AaI2gRu4UMfrhIIc,youtube#video,YGPP1BjvgCU,2023-02-02T10:48:45Z,UC8_stYIOMyTQs_cEKkopaFQ,"Let&#39;s do an Eye Mask, and Sheet Mask #revi...",Stylevana Makeup and Skincare. I have a code y...,https://i.ytimg.com/vi/YGPP1BjvgCU/default.jpg,120,90,https://i.ytimg.com/vi/YGPP1BjvgCU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/YGPP1BjvgCU/hqdefault.jpg,480,360,Life with Christine,none,2023-02-02T10:48:45Z
944,youtube#searchResult,r1fvc4bljLvRM3LA8vuAlRthlKM,youtube#video,7JlZiPmfJUM,2023-02-20T08:03:00Z,UCnaasPYchRGCIsh9sCoTEhA,come on rest~ 4F solaria hotel #myeongdongkyoj...,NaN,https://i.ytimg.com/vi/7JlZiPmfJUM/default.jpg,120,90,https://i.ytimg.com/vi/7JlZiPmfJUM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/7JlZiPmfJUM/hqdefault.jpg,480,360,MyeongDongHairSalon,none,2023-02-20T08:03:00Z
945,youtube#searchResult,XqD468Q_82WhDFB0_OkgigCleIU,youtube#video,5c1f97i8XZM,2023-02-13T03:40:15Z,UC8J8CPTDZsFRh9dRfm-xayg,Are you into Vegan Products? #skincareroutine ...,"[ Unboxing ] Melixir, is a Vegan Skincar

In [20]:
#19.1 Запросы БЕЗ включения аргумента order, но с учетом даты
# Первый заход:
request = youtube.search().list(
        part ="snippet",
        maxResults=50,
        q = query,
        type = type_query,
        publishedAfter=publishedAfter,
        publishedBefore=publishedBefore
)
response = request.execute()  
   
df_supplemented = pd.json_normalize(response['items'])
   
    # Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            type = type_query,
            pageToken = response['nextPageToken'],
            publishedAfter=publishedAfter,
            publishedBefore=publishedBefore
    )
    response = request.execute()  
    
    print(f'Итерация №{i}, "date" {publishedAfter[:11]} - {publishedBefore[:11]}')
    i += 1
        
    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])
    
    

print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates(f'id.{type_query}Id'))}")

Итерация №0, "date" 2020-01-01T - 2022-01-01T
Итерация №1, "date" 2020-01-01T - 2022-01-01T
Итерация №2, "date" 2020-01-01T - 2022-01-01T
Итерация №3, "date" 2020-01-01T - 2022-01-01T
Итерация №4, "date" 2020-01-01T - 2022-01-01T
Итерация №5, "date" 2020-01-01T - 2022-01-01T
Итерация №6, "date" 2020-01-01T - 2022-01-01T
Итерация №7, "date" 2020-01-01T - 2022-01-01T
Итерация №8, "date" 2020-01-01T - 2022-01-01T
Итерация №9, "date" 2020-01-01T - 2022-01-01T
Искомых объектов 1000000 а найденных БЕЗ включения каких-либо значений аргумента order 481


In [25]:
# 19.2
# Цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи:
# Думаю, можно не выполнять, так как большинство включено в искомые объекты

In [21]:
# 21 Проверка совпадения двух столбцов из выдачи:
df_supplemented = pd.read_excel(DATA_PATH / f'{type_query.capitalize()}_Not_sorted+Sorted.xlsx', index_col=0)
# возврат к выдаче без учета date; указание, что индекс в первом столбце

display(
    df_supplemented,
    # df_supplemented['snippet.publishedAt'] == df_supplemented['snippet.publishTime'],
    f"Совпадений двух столбцов: {sum(df_supplemented['snippet.publishedAt'] == df_supplemented['snippet.publishTime'])}"
    )

,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
kind,,,,,,,,,,,,,,,,,,,
youtube#searchResult,xUPXCyswbDfQ0O_cEo1y9jBHY-I,youtube#video,AOVTUmDU7Bs,2018-07-12T06:00:04Z,UCgln4bTPN2cXnAjXVgM-Cfw,Annoying Moments While Getting Ready ENG SUB •...,ANNOYING MOMENTS WHILE GETTING READY!! UGH!! M...,https://i.ytimg.com/vi/AOVTUmDU7Bs/default.jpg,120,90,https://i.ytimg.com/vi/AOVTUmDU7Bs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AOVTUmDU7Bs/hqdefault.jpg,480,360,DGG,none,2018-07-12T06:00:04Z
youtube#searchResult,ry0zlUewmQ1_rgpvH5UeqQjwyOE,youtube#video,oNJWU8cm0eU,2020-08-25T06:38:17Z,UChhOtjq-3QyyLmP2jv9amrg,Rhythm of K-Beauty,Rhythm of K-Beauty Want to learn about Korean ...,https://i.ytimg.com/vi/oNJWU8cm0eU/default.jpg,120,90,https://i.ytimg.com/vi/oNJWU8cm0eU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/oNJWU8cm0eU/hqdefault.jpg,480,360,Imagine Your Korea,none,2020-08-25T06:38:17Z
youtube#searchResult,FAWm2FVeXAKFMi8_98auJXmRjZo,youtube#video,fgeqSoLgccU,2018-07-10T06:00:30Z,UCgln4bTPN2cXnAjXVgM-Cfw,Short Hair VS Long Hair ENG SUB • dingo kbeauty,WORLD'S FIRST PROBLEM This is #1 thing I ask m...,https://i.ytimg.com/vi/fgeqSoLgccU/default.jpg,120,90,https://i.ytimg.com/vi/fgeqSoLgccU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/fgeqSoLgccU/hqdefault.jpg,480,360,DGG,none,2018-07-10T06:00:30Z
youtube#searchResult,6Lv9z7Od-sjYY5pxt54NruY3weY,youtube#video,gvsV9ds9uFk,2022-04-15T04:52:25Z,UCJ9E_Ob8i8-2cyd67mbkRfw,Pretty duo 💓 #lipbalm #kbeauty,NaN,https://i.ytimg.com/vi/gvsV9ds9uFk/default.jpg,120,90,https://i.ytimg.com/vi/gvsV9ds9uFk/mqdefault.jpg,320,180,https://i.ytimg.com/vi/gvsV9ds9uFk/hqdefault.jpg,480,360,Kerbom3,none,2022-04-15T04:52:25Z
youtube#searchResult,a5HhfVq4RM98pKJBdn9oqGBOICE,youtube#video,o5hc2PaPFZ8,2021-09-24T16:54:57Z,UCJ9E_Ob8i8-2cyd67mbkRfw,#kajabeauty #kbeauty #kbeautyreview #review #a...,NaN,https://i.ytimg.com/vi/o5hc2PaPFZ8/default.jpg,120,90,https://i.ytimg.com/vi/o5hc2PaPFZ8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/o5hc2PaPFZ8/hqdefault.jpg,480,360,Kerbom3,none,2021-09-24T16:54:57Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
youtube#searchResult,yrLbtJF5skD__0R2g-TTUhPyM8c,youtube#video,DsOeHUYwjpw,2023-02-02T21:22:27Z,UCpjNgaKGfDCGPbP7JmgjZHw,Trying one of the TOP sunscreens in Korea 🫣 #k...,Follow for more glow ✨ Instagram: https://www....,https://i.ytimg.com/vi/DsOeHUYwjpw/default.jpg,120,90,https://i.ytimg.com/vi/DsOeHUYwjpw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/DsOeHUYwjpw/hqdefault.jpg,480,360,vpofglow,none,2023-02-02T21:22:27Z
youtube#searchResult,xW4tKawA4C8AaI2gRu4UMfrhIIc,youtube#video,YGPP1BjvgCU,2023-02-02T10:48:45Z,UC8_stYIOMyTQs_cEKkopaFQ,"Let&#39;s do an Eye Mask, and Sheet Mask #revi...",Stylevana Makeup and Skincare. I have a code y...,https://i.ytimg.com/vi/YGPP1BjvgCU/default.jpg,120,90,https://i.ytimg.com/vi/YGPP1BjvgCU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/YGPP1BjvgCU/hqdefault.jpg,480,360,Life with Christine,none,2023-02-02T10:48:45Z
youtube#searchResult,r1fvc4bljLvRM3LA8vuAlRthlKM,youtube#video,7JlZiPmfJUM,2023-02-20T08:03:00Z,UCnaasPYchRGCIsh9sCoTEhA,come on rest~ 4F solaria hotel #myeongdongkyoj...,NaN,https://i.ytimg.com/vi/7JlZiPmfJUM/default.jpg,120,90,https://i.ytimg.com/vi/7JlZiPmfJUM/mqdefault.jpg,320,180,https://i.ytimg.com/vi/7JlZiPmfJUM/hqdefault.jpg,480,360,MyeongDongHairSalon,none,2023-02-20T08:03:00Z


'Совпадений двух столбцов: 947'

In [22]:
# 22 Проверка индексирования содержимого столбца snippet.publishedAt и строчки №1:
df_supplemented['snippet.publishedAt'][1][:4]

'2020'

In [23]:
# 23 Lambda-функция для перезаписи в столбец snippet.publishedAt только года:
df_supplemented['snippet.publishedAt'] = df_supplemented['snippet.publishedAt'].apply(lambda yyyy: int(yyyy[:4]))
df_supplemented['snippet.publishedAt']

kind
youtube#searchResult    2018
youtube#searchResult    2020
youtube#searchResult    2018
youtube#searchResult    2022
youtube#searchResult    2021
                        ... 
youtube#searchResult    2023
youtube#searchResult    2023
youtube#searchResult    2023
youtube#searchResult    2023
youtube#searchResult    2023
Name: snippet.publishedAt, Length: 947, dtype: int64

In [32]:
# 24 Поиск самого раннего года:
min_date = min(df_supplemented['snippet.publishedAt'])
max_date = max(df_supplemented['snippet.publishedAt'])

print("Минимальный год с видео", min_date)
print("Максимальный год с видео", max_date)

Минимальный год с видео 2016
Максимальный год с видео 2023


In [33]:
#25.1 Запросы БЕЗ включения аргумента order, но с учетом даты

# На всякий случай получим следующий ключ

i = 0
year = 2018

while year < max_date:
     
    # Первый заход С включением аргумента date:
    request = youtube.search().list(
        part ="snippet",
        maxResults=50,
        q = query,
        regionCode ="RU",
        type = type_query,
        publishedAfter=f'{year}-01-01T00:00:00Z'
    )
    response = request.execute()  
   
    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])
   
    # Цикл для прохода по всем следующим страницам с выдачей:
    while 'nextPageToken' in response.keys():
        request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            regionCode ="RU",
            type = type_query,
            pageToken = response['nextPageToken'],
            publishedAfter=f'{year}-01-01T00:00:00Z'
        )
        response = request.execute()  
    
        # Визуализация процесса:
        print(f'Итерация №{i}, "date" {year}')
        i += 1
        
        df_additional = pd.json_normalize(response['items'])
        df_supplemented = pd.concat([df_supplemented, df_additional])
    

    print(f"Искомых объектов с {year} года {response['pageInfo']['totalResults']}, \
      а найденных {len(df_supplemented.drop_duplicates(f'id.{type_query}Id'))}")

    year += 1

Итерация №0, "date" 2018
Итерация №1, "date" 2018
Итерация №2, "date" 2018
Итерация №3, "date" 2018
Итерация №4, "date" 2018
Итерация №5, "date" 2018
Итерация №6, "date" 2018
Итерация №7, "date" 2018
Итерация №8, "date" 2018
Итерация №9, "date" 2018
Итерация №10, "date" 2018
Искомых объектов с 2018 года 1000000,       а найденных 1021
Итерация №11, "date" 2019
Итерация №12, "date" 2019
Итерация №13, "date" 2019
Итерация №14, "date" 2019
Итерация №15, "date" 2019
Итерация №16, "date" 2019
Итерация №17, "date" 2019
Итерация №18, "date" 2019
Итерация №19, "date" 2019
Итерация №20, "date" 2019
Итерация №21, "date" 2019
Искомых объектов с 2019 года 1000000,       а найденных 1054
Итерация №22, "date" 2020
Итерация №23, "date" 2020
Итерация №24, "date" 2020
Итерация №25, "date" 2020
Итерация №26, "date" 2020
Итерация №27, "date" 2020
Итерация №28, "date" 2020
Итерация №29, "date" 2020
Искомых объектов с 2020 года 1000000,       а найденных 1093
Итерация №30, "date" 2021
Итерация №31, "date" 

In [35]:
# 20
# Удаление дубликатов каналов (по их ID) и запись в Excel:
df_supplemented.index = range(1,len(df_supplemented)+1) # сквозной индекс для итоговой таблицы
df_supplemented = df_supplemented.drop_duplicates(f'id.{type_query}Id')
display(df_supplemented)


,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime,kind
1,xUPXCyswbDfQ0O_cEo1y9jBHY-I,youtube#video,AOVTUmDU7Bs,2018,UCgln4bTPN2cXnAjXVgM-Cfw,Annoying Moments While Getting Ready ENG SUB •...,ANNOYING MOMENTS WHILE GETTING READY!! UGH!! M...,https://i.ytimg.com/vi/AOVTUmDU7Bs/default.jpg,120,90,https://i.ytimg.com/vi/AOVTUmDU7Bs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/AOVTUmDU7Bs/hqdefault.jpg,480,360,DGG,none,2018-07-12T06:00:04Z,NaN
2,ry0zlUewmQ1_rgpvH5UeqQjwyOE,youtube#video,oNJWU8cm0eU,2020,UChhOtjq-3QyyLmP2jv9amrg,Rhythm of K-Beauty,Rhythm of K-Beauty Want to learn about Korean ...,https://i.ytimg.com/vi/oNJWU8cm0eU/default.jpg,120,90,https://i.ytimg.com/vi/oNJWU8cm0eU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/oNJWU8cm0eU/hqdefault.jpg,480,360,Imagine Your Korea,none,2020-08-25T06:38:17Z,NaN
3,FAWm2FVeXAKFMi8_98auJXmRjZo,youtube#video,fgeqSoLgccU,2018,UCgln4bTPN2cXnAjXVgM-Cfw,Short Hair VS Long Hair ENG SUB • dingo kbeauty,WORLD'S FIRST PROBLEM This is #1 thing I ask m...,https://i.ytimg.com/vi/fgeqSoLgccU/default.jpg,120,90,https://i.ytimg.com/vi/fgeqSoLgccU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/fgeqSoLgccU/hqdefault.jpg,480,360,DGG,none,2018-07-10T06:00:30Z,NaN
4,6Lv9z7Od-sjYY5pxt54NruY3weY,youtube#video,gvsV9ds9uFk,2022,UCJ9E_Ob8i8-2cyd67mbkRfw,Pretty duo 💓 #lipbalm #kbeauty,NaN,https://i.ytimg.com/vi/gvsV9ds9uFk/default.jpg,120,90,https://i.ytimg.com/vi/gvsV9ds9uFk/mqdefault.jpg,320,180,https://i.ytimg.com/vi/gvsV9ds9uFk/hqdefault.jpg,480,360,Kerbom3,none,2022-04-15T04:52:25Z,NaN
5,a5HhfVq4RM98pKJBdn9oqGBOICE,youtube#video,o5hc2PaPFZ8,2021,UCJ9E_Ob8i8-2cyd67mbkRfw,#kajabeauty #kbeauty #kbeautyreview #review #a...,NaN,https://i.ytimg.com/vi/o5hc2PaPFZ8/default.jpg,120,90,https://i.ytimg.com/vi/o5hc2PaPFZ8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/o5hc2PaPFZ8/hqdefault.jpg,480,360,Kerbom3,none,2021-09-24T16:54:57Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,C5ZSuAtTkMrjeBvqJGVNQxk_HH8,youtube#video,Hu61nOX01-0,2022-11-12T00:54:24Z,UC4hwR9831LIKr9wKQOO_ZYA,How do get IDOL’S CURTAIN BANGS? CELEBRITY #sh...,,https://i.ytimg.com/vi/Hu61nOX01-0/default.jpg,120,90,https://i.ytimg.com/vi/Hu61nOX01-0/mqdefault.jpg,320,180,https://i.ytimg.com/vi/Hu61nOX01-0/hqdefault.jpg,480,360,"Kyurin Diary, Multilingual Traveler",none,2022-11-12T00:54:24Z,youtube#searchResult
1193,v90hbzRjX48mvAZCws9D9BTuliQ,youtube#video,vQ2EZey0n_M,2023-01-31T10:56:50Z,UCQTWwkU4zxqmxv-ywGkGPSA,#exosome ampoule korean skin care ~ #피부역행템 #kb...,,https://i.ytimg.com/vi/vQ2EZey0n_M/default.jpg,120,90,https://i.ytimg.com/vi/vQ2EZey0n_M/mqdefault.jpg,320,180,https://i.ytimg.com/vi/vQ2EZey0n_M/hqdefault.jpg,480,360,Joyful_yeonsu,none,2023-01-31T10:56:50Z,youtube#searchResult
1194,Frp6d0X8Z2VsF1dZvT4WL66U8U4,youtube#video,6A5uJXxPbQs,2022-10-15T06:06:44Z,UCdIEHk1dbkXvWPh8LvyJQgg,Makeup look for selfies #shorts #makeup #makeu...,,https://i.ytimg.com/vi/6A5uJXxPbQs/default.jpg,120,90,https://i.ytimg.com/vi/6A5uJXxPbQs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/6A5uJXxPbQs/hqdefault.jpg,480,360,minana,none,2022-10-15T06:06:44Z,youtube#searchResult
1195,SvvmimKVFGdfwOpVyswXecrzVuE,youtube#video,lP4oNCbXmUg,2023-02-25T20:18:44Z,UCV5d39R6e3nfAaW5aldP_ww,Types of Vitamin C Serums #jumiso #serums #kbe...,,https://i.ytimg.com/vi/lP4oNCbXmUg/default.jpg,120,90,https://i.ytimg.com/vi/lP4oNCbXmUg/mqdefault.jpg,320,180,https://i.ytimg.com/vi/lP4oNCbXmUg/hqdefault.jpg,480,360,fightingmothernature,none,2023-02-25T20:18:44Z,youtube#searchResult


In [36]:
import os

# сохранение файлов в специальной директории

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

df_supplemented.to_excel(DATA_PATH / f'{query}_{type_query.capitalize()}_Not_sorted+Sorted+Date.xlsx')